In [ ]:
import torch, numpy as np, scipy.sparse as sp
import torch.optim as optim, torch.nn.functional as F
import torch_sparse
import pickle
import config
import random
from torch_scatter import scatter
from collections import defaultdict
import time
import torch.nn as nn, torch.nn.functional as F
import math
from torch_geometric.utils import softmax
import sklearn.metrics
import os
from torchinfo import summary
args = config2.parse()
#device = torch.device('cuda:'+args.cuda if torch.cuda.is_available() else 'cpu')

In [ ]:
device = torch.device('cuda:1')

In [ ]:
com_dict_train = defaultdict(set) 
with open('1101_1108/c_u_train.txt','r') as file:
    length = 0
    while 1:
        line = file.readline()
        if(line==''):
            break
        n = len(line)
        s = line[0:n-1].split(',')
        #print(s)
        length += (len(s)-1)
        for j in range(len(s)-1):
            #print(s[0],s[j+1])
            com_dict_train[s[0]].add(int(s[j+1]))

In [ ]:
user_com_dict = {}
with open('1101_1108/c_u_train.txt','r') as file:
    while 1:
        line = file.readline()
        if(line==''):
            break
        n = len(line)
        s = line[0:n-1].split(',')
        for j in range(len(s)-1):
            user_com_dict[int(s[j+1])] = int(s[0])

In [ ]:
f = open("1101_1108/com_user_train.pkl", "wb")
pickle.dump(com_dict_train, f)
f.close()

In [ ]:
com_dict_test = defaultdict(set) 
with open('1101_1108/c_u_test.txt','r') as file:
    length = 0
    while 1:
        line = file.readline()
        if(line==''):
            break
        n = len(line)
        s = line[0:n-1].split(',')
        length+=(len(s)-1)
        for j in range(len(s)-1):
            #print(s[0],s[j+1])
            com_dict_test[s[0]].add(int(s[j+1]))

In [ ]:
f = open("1101_1108/com_user_test.pkl", "wb")
pickle.dump(com_dict_test, f)
f.close()

In [ ]:
dict_1={}
i=0
with open('1101_1108/c_u_train.txt','r') as file:
    while 1:
        line = file.readline()
        if(line==''):
            break
        n = len(line)
        s = line[:n-1].split(',')
        list_1 = []
        list_1.append(int(s[0])+args.user_number)
        for j in range(len(s)-1):
            list_1.append(int(s[j+1]))
        dict_1[i]=set(list_1)
        i+=1

In [ ]:
f = open("1101_1108/com_user_train_edge.pkl", "wb")
pickle.dump(dict_1, f)
f.close()

In [ ]:
dict_2={}
i=0
with open('1101_1108/u_a.txt','r') as file:
    while 1:
        line = file.readline()
        if(line==''):
            break
        n = len(line)
        s = line[:n-1].split(',')
        list_1 = []
        list_1.append(int(s[0]))
        if(user_com_dict.__contains__(int(s[0]))):
            #print(s[0])
            list_1.append(int(user_com_dict[int(s[0])])+args.user_number)
        for j in range(len(s)-1):
            list_1.append(int(s[j+1])+args.user_number+args.community_number)
        dict_2[i]=set(list_1)
        i+=1

In [ ]:
f = open("1101_1108/user_attr_edge.pkl", "wb")
pickle.dump(dict_2, f)
f.close()

In [ ]:
mat_cu = []
i=0
with open('1101_1108/c_u_train.txt','r') as file:
    while 1:
        line = file.readline()
        if(line==''):
            break
        n = len(line)
        s = line[:n-1].split(',')
        list_1 = [0]*args.user_number
        for j in range(1,len(s)-1):
            list_1[int(s[j])]=1
        mat_cu.append(list_1)

In [ ]:
mat_ua = []
with open('1101_1108/u_a.txt','r') as file:
    while 1:
        line = file.readline()
        if(line==''):
            break
        n = len(line)
        s = line[:n-1].split(',')
        list_1 = [0]*args.attribute_number
        for j in range(1,len(s)-1):
            list_1[int(s[j])]=1
        mat_ua.append(list_1)

In [ ]:
mat_ca = []
with open('1101_1108/c_a.txt','r') as file:
    while 1:
        line = file.readline()
        if(line==''):
            break
        n = len(line)
        s = line[:n-1].split(',')
        list_1 = [0]*19
        for j in range(1,len(s)-1):
            list_1[int(s[j])]=1
        mat_ca.append(list_1)

In [ ]:
mat_cu = torch.tensor(mat_cu,  dtype=torch.float)
mat_ua = torch.tensor(mat_ua,  dtype=torch.float)
mat_ca = torch.tensor(mat_ca,  dtype=torch.float)

In [ ]:
# gpu, seed
torch.manual_seed(args.seed)
np.random.seed(args.seed)

def get_data(args):
    dataset = args.dataset
    print('dataset name: ', dataset)
    
    #node numbers
    user_number = args.user_number
    train_user_number = args.train_user_number
    test_user_number = args.test_user_number
    community_number=args.community_number
    attribute_number=args.attribute_number
    day_point_number=args.day_point_number
    node_number = community_number+attribute_number+train_user_number+test_user_number

    #edge and edge numbers
    com_user = open(dataset + "/com_user_train_edge.pkl",'rb+')  ## community1: user1, user2, user3,...
    com_user_edge = pickle.load(com_user)
    com_user_edge_num=len(com_user_edge)
    args.com_user_edge_num=com_user_edge_num
    print("community hyperedge number is:", com_user_edge_num)

    user_attr = open(dataset+"/user_attr_edge.pkl",'rb+')  ## user1: community, attr1, attr2, attr3,...
    user_attr_edge=pickle.load(user_attr)
    user_attr_edge_num=len(user_attr_edge)
    args.user_attr_edge_num=user_attr_edge_num
    print("user hyperedge is:", user_attr_edge_num)
    
    
    

    all_hyper_edge = []


    #train data and test data
    com_user_train = open(dataset + "/com_user_train.pkl",'rb+')
    com_user_train = pickle.load(com_user_train)
    com_user_test = open(dataset + "/com_user_test.pkl",'rb+')
    com_user_test = pickle.load(com_user_test)
    
    
    #train and test data
    K=args.negative_K
    com_user_list_train = []
    for i in com_user_train.keys():
        for j in com_user_train[i]:
            com_user_list_train.append([j, int(i)+args.user_number])
    com_user_list_train_true = com_user_list_train.copy()

    com_user_list_test = []
    for i in com_user_test.keys():
        for j in com_user_test[i]:
            com_user_list_test.append([j, int(i)+args.user_number])
    com_user_list_test_true = com_user_list_test.copy()

    for i in range(len(com_user_list_test)):
        neg = random.randint(args.user_number, args.user_number+args.community_number-1)
        while 1:
            if (neg == com_user_list_test[i][1]):
                neg = random.randint(args.user_number, args.user_number+args.community_number-1)
            else:
                break
        com_user_list_test.append([com_user_list_test[i][0], neg])
    

    for i in range(len(com_user_list_train)):
        for j in range(K):
            neg = random.randint(args.user_number, args.user_number+args.community_number-1)
            while 1:
                if (neg == com_user_list_train[i][1]):
                    neg = random.randint(args.user_number, args.user_number+args.community_number-1)
                else:
                    break
            com_user_list_train.append([com_user_list_train[i][0], neg])


    
    test_label=[]
    for i in range(2* len(com_user_list_test_true)):
        if i <len(com_user_list_test_true):
            test_label.append(1.0)
        else:
            test_label.append(0.0)
    test_label=np.array(test_label)
            
    train_label=[]
    for i in range((1+K)* len(com_user_list_train_true)):
        if i <len(com_user_list_train_true):
            train_label.append(1.0)
        else:
            train_label.append(0.0)
            
            
            
            
    

    G={}
    G['community_user'] = com_user_edge
    G['user_attribute'] = user_attr_edge

    return   G, com_user_list_test, com_user_list_train, test_label, train_label


In [ ]:
torch.manual_seed(args.seed)
np.random.seed(args.seed)

model_name = args.model_name

node_type=['user','community','attribute']
edge_type=['community_user','user_attribute']

G, com_user_list_test, com_user_list_train, test_label, train_label = get_data(args)

dataset name:  1101_1108
community hyperedge number is: 453
user hyperedge is: 8442


In [ ]:
'''
f = open("1101_1108/com_user_list_test.pkl", "wb")
pickle.dump(com_user_list_test, f)
f.close()

f = open("1101_1108/com_user_list_train.pkl", "wb")
pickle.dump(com_user_list_train, f)
f.close()

f = open("1101_1108/test_label.pkl", "wb")
pickle.dump(test_label, f)
f.close()

f = open("1101_1108/train_label.pkl", "wb")
pickle.dump(train_label, f)
f.close()
'''

'\nf = open("1101_1108/com_user_list_test.pkl", "wb")\npickle.dump(com_user_list_test, f)\nf.close()\n\nf = open("1101_1108/com_user_list_train.pkl", "wb")\npickle.dump(com_user_list_train, f)\nf.close()\n\nf = open("1101_1108/test_label.pkl", "wb")\npickle.dump(test_label, f)\nf.close()\n\nf = open("1101_1108/train_label.pkl", "wb")\npickle.dump(train_label, f)\nf.close()\n'

In [ ]:

fp = open("1101_1108/com_user_list_test.pkl", "rb+")
com_user_list_test = pickle.load(fp)
fp.close()
fp = open("1101_1108/com_user_list_train.pkl", "rb+")
com_user_list_train = pickle.load(fp)
fp.close()
fp = open("1101_1108/test_label.pkl", "rb+")
test_label = pickle.load(fp)
fp.close()
fp = open("1101_1108/train_label.pkl", "rb+")
train_label = pickle.load(fp)
fp.close()


In [ ]:
com_user_tensor_test = (torch.tensor(np.array(com_user_list_test),dtype=torch.long)).t().contiguous()
com_user_tensor_train = (torch.tensor(np.array(com_user_list_train),dtype=torch.long)).t().contiguous()
train_label=torch.tensor(np.array(train_label),dtype=torch.float32).to(device)

In [ ]:
def initial(G, args, node_type, edge_type, mat_cu, mat_ua, mat_ca, unseen=None):

    G_all={}
    z=0
    for i in edge_type:
        for j in range(len(G[i])):
            G_all[z] = G[i][j]
            z+=1

    G = G_all.copy()
    if unseen is not None:
        unseen = set(unseen)
        # remove unseen nodes
        for e, vs in G.items():
            G[e] =  list(set(vs) - unseen)
    node_number= args.user_number+ args.community_number + args.attribute_number
    args.node_num = node_number
    N, M = node_number, len(G)

    indptr, indices, data = [0], [], []
    for e, vs in G.items():
        indices += vs
        data += [1] * len(vs)
        indptr.append(len(indices))
    H = sp.csc_matrix((data, indices, indptr), shape=(N, M), dtype=int).tocsr()

    (row, col), value = torch_sparse.from_scipy(H)
    V, E = row, col
    args.edge_num=max(E)+1

    edge_input_length= [args.user_attr_edge_num, args.com_user_edge_num]
    node_input_length = [args.user_number, args.community_number, args.attribute_number]

    V_raw_index_type=[0 for i in range(args.user_number)]+ [1 for i in range(args.community_number)]+[2 for i in range(args.attribute_number)]
    args.V_raw_index_type=torch.tensor(V_raw_index_type, dtype=torch.long)
    E_raw_index_type=[0 for i in range(args.user_attr_edge_num)]+ [1 for i in range(args.com_user_edge_num)]
    args.E_raw_index_type=torch.tensor(E_raw_index_type, dtype=torch.long)

    args.edge_type=edge_type
    args.node_type=node_type

    a=0
    edge_input_length_raw = []
    for i in range(len(edge_input_length)):
        edge_input_length_raw.append(a+edge_input_length[i])
        a=a + edge_input_length[i]
    args.edge_input_length_raw = edge_input_length_raw

    b=0
    node_input_length_raw=[]
    for i in range(len(node_input_length)):
        node_input_length_raw.append(b+node_input_length[i])
        b= b+ node_input_length[i]
    args.node_input_length_raw = node_input_length_raw

    V_class=[]
    V_class_index_0, V_class_index_1, V_class_index_2=[],[],[]
    for i in range(V.shape[0]):
        if V[i] <node_input_length_raw[0]:
            V_class.append(0) #user
            V_class_index_0.append(i)
        elif node_input_length_raw[0]<= V[i] <node_input_length_raw[1]:
            V_class.append(1)#community
            V_class_index_1.append(i)
        elif node_input_length_raw[1]<= V[i] <node_input_length_raw[2]:
            V_class.append(2)#attribute
            V_class_index_2.append(i)

    E_class=[]
    E_class_index_0, E_class_index_1=[],[]
    for i in range(E.shape[0]):
        if E[i]<edge_input_length_raw[0]:
            E_class.append(0) #user-attr
            E_class_index_0.append(i)
        elif edge_input_length_raw[0]<=E[i]<edge_input_length_raw[1]:
            E_class.append(1) #com-user
            E_class_index_1.append(i)


    args.V_class=torch.tensor(V_class,dtype=torch.long)
    args.E_class=torch.tensor(E_class,dtype=torch.long)

    args.V_class_index_0=torch.tensor(V_class_index_0,dtype=torch.long)
    args.V_class_index_1=torch.tensor(V_class_index_1,dtype=torch.long)
    args.V_class_index_2=torch.tensor(V_class_index_2,dtype=torch.long)

    args.E_class_index_0=torch.tensor(E_class_index_0,dtype=torch.long)
    args.E_class_index_1=torch.tensor(E_class_index_1,dtype=torch.long)

    args.V=V
    args.E=E
    V, E = V.to(device), E.to(device)
    mat_ca = mat_ca.to(device)
    print(device)
    print(mat_ca.device)
    args.edge_input_length=edge_input_length_raw
    args.node_input_length=node_input_length_raw

    args.dataset_dict={'hypergraph':G,'n':N,'features':torch.randn(N,args.input_dim)}

    nhid = args.nhid
    nhead = args.nhead
    dropout_ratio = args.dropout
    E_class_index=torch.unsqueeze(torch.cat((args.E_class_index_0,args.E_class_index_1) ,0),1)
    args.E_class_index  =E_class_index.repeat(1,nhead)
    V_class_index=torch.unsqueeze(torch.cat((args.V_class_index_0,args.V_class_index_1,args.V_class_index_2) ,0) ,1)
    args.V_class_index= V_class_index.repeat(1,nhead)


    model = HATT(args, args.input_dim, nhid, args.out_dim, nhead, dropout_ratio, V, E,  edge_type, node_type, mat_cu, mat_ua, mat_ca)
    optimiser = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.wd)
    model.to(device)
    return model, optimiser, G

In [ ]:
def glorot(tensor):
    if tensor is not None:
        stdv = math.sqrt(6.0 / (tensor.size(-2) + tensor.size(-1)))
        tensor.data.uniform_(-stdv, stdv)
        
        
def normalize_l2(X):
    """Row-normalize  matrix"""
    rownorm = X.detach().norm(dim=1, keepdim=True)
    scale = rownorm.pow(-1)
    scale[torch.isinf(scale)] = 0.
    X = X * scale
    return X

class hattConv(nn.Module):
    def __init__(self, args, in_channels, out_channels, heads, negative_slope=0.2, device=device, dropout = 0):
        super().__init__()
        self.heads = heads
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.dropout = dropout

        self.weight = nn.Parameter(torch.Tensor(in_channels, heads * out_channels))
        self.bias = nn.Parameter(torch.Tensor(heads * out_channels))
        #self.W = nn.Linear(in_channels, heads * out_channels, bias=True)
        #stdv = math.sqrt(6.0/(self.in_channels+self.heads * self.out_channels))
        #nn.init.uniform_(self.W.weight.data, -stdv, stdv)
        #nn.init.uniform_(self.W.bias.data, -stdv, stdv)
        
        self.att_v_user=nn.Parameter(torch.Tensor(1, heads, out_channels))
        self.att_v_community = nn.Parameter(torch.Tensor(1, heads, out_channels))
        self.att_v_attribute = nn.Parameter(torch.Tensor(1, heads, out_channels))

        self.att_e_com = nn.Parameter(torch.Tensor(1, heads, out_channels))    #每个社团的注意力机制
        self.att_e_user = nn.Parameter(torch.Tensor(1, heads, out_channels))   #每个user的注意力机制
        
        self.leaky_relu = nn.LeakyReLU(negative_slope)
        self.args = args
        self.edge_num = args.edge_num
        self.node_num = args.node_num
        self.edge_type = args.edge_type
        self.node_type = args.node_type
        self.edge_input_length = args.edge_input_length
        self.node_input_length = args.node_input_length
        self.reset_parameters()

        self.V_raw_index_type= (args.V_raw_index_type).to(device)
        self.V_class=(args.V_class).to(device)
        self.E_class=(args.E_class).to(device)
        self.V_class_index=(args.V_class_index).to(device)
        self.E_class_index=(args.E_class_index).to(device)
        self.V_class_index_0 =(args.V_class_index_0).to(device)
        self.V_class_index_1 =(args.V_class_index_1).to(device)
        self.V_class_index_2 =(args.V_class_index_2).to(device)
        self.E_class_index_0 =(args.E_class_index_0).to(device)
        self.E_class_index_1 =(args.E_class_index_1).to(device)
        
        self.relu = nn.ReLU()

    def __repr__(self):
        return '{}({}, {}, heads={})'.format(self.__class__.__name__,
                                             self.in_channels, self.out_channels, self.heads)
    
    def reset_parameters(self):
        stdv =  math.sqrt(6./(self.in_channels+self.heads * self.out_channels))
        self.weight.data.uniform_(-stdv, stdv)
        self.bias.data.uniform_(-stdv, stdv)
        glorot(self.att_v_user)
        glorot(self.att_v_community)
        glorot(self.att_v_attribute)
        glorot(self.att_e_com)
        glorot(self.att_e_user)

        
    def forward(self, X, vertex, edges):
        H, C = self.heads, self.out_channels
        X0 = X.matmul(self.weight) + self.bias
        X = X0.view(self.node_num, H, C)
        Xve = X[vertex]
        X = Xve
        Y_e_0 = (torch.index_select(X, 0, self.E_class_index_0) * self.att_e_com).sum(-1)
        Y_e_1 = (torch.index_select(X, 0, self.E_class_index_1) * self.att_e_user).sum(-1)
        Y_e = torch.cat((Y_e_0, Y_e_1), 0)
        #print('1',Y_e.shape)
        beta_v = torch.gather(Y_e, 0, self.E_class_index)
        beta = self.leaky_relu(beta_v)
        beta = softmax(beta, edges, num_nodes=self.edge_num)
        beta = beta.unsqueeze(-1)
        Ye = Xve * beta
        #print('2',Ye.shape)
        Ye = (scatter(Ye, edges, dim=0, reduce='sum', dim_size=self.edge_num))
        #print('3',Ye.shape)
        Ye = F.dropout(Ye, self.dropout, training=self.training)
        Y_out = Ye.view(self.edge_num, H * C)
        Y_out = self.relu(Y_out)
        Ye = Ye[edges]
        Y = Ye
        X_ve_0 = (torch.index_select(Y, 0, self.V_class_index_0) * self.att_v_user).sum(-1)
        X_ve_1 = (torch.index_select(Y, 0, self.V_class_index_1) * self.att_v_attribute).sum(-1)
        X_ve_2 = (torch.index_select(Y, 0, self.V_class_index_2) * self.att_v_community).sum(-1)
        X_ve = torch.cat((X_ve_0, X_ve_1, X_ve_2), 0)
        alpha_e = torch.gather(X_ve, 0, self.V_class_index)
        alpha = self.leaky_relu(alpha_e)
        alpha = softmax(alpha, vertex, num_nodes=self.node_num)
        alpha = alpha.unsqueeze(-1)
        Xve = Ye * alpha
        Xve = scatter(Xve, vertex, dim=0, reduce='sum', dim_size=self.node_num)  # [N, H, C]
        Xve = F.dropout(Xve, self.dropout, training=self.training)
        X_out = Xve.view(self.node_num, H * C)
        X_out = self.relu(X_out)
        return X_out, Y_out

    


class HATT(nn.Module):
    def __init__(self, args, nfeat, nhid, out_dim, nhead, drop_rate, V, E, edge_type, node_type, mat_cu, mat_ua, mat_ca):

        super().__init__()
        self.conv_in = hattConv(args, nfeat, nhid, heads=nhead, device=device, dropout = drop_rate)
        self.conv_out = hattConv(args, nhid*nhead, nhid, heads=args.out_nhead, device=device, dropout = 0)
        
        attr_feature = torch.FloatTensor(args.attribute_number, nfeat)
        nn.init.xavier_uniform_(attr_feature)
        user_feature = torch.matmul(mat_ua, attr_feature)
        com_feature = torch.matmul(mat_cu, user_feature)
        X = [user_feature, com_feature, attr_feature]
        self.X = nn.Parameter(torch.cat((X), 0))
        self.V = V
        self.E = E
        self.edge_type = edge_type
        self.node_type = node_type
        self.mat_ca = mat_ca
        self.lin_out = nn.Linear(nhid *args.out_nhead, out_dim, bias=True)
        self.com_embed = nn.Linear(mat_ca.shape[1], out_dim, bias=True)

    
    def forward(self):

        V, E = (self.V) , (self.E)
        E_com = self.com_embed(self.mat_ca)
        X_1,Y_1 = self.conv_in(self.X, V, E)
        X_1,Y_1 = self.conv_out(X_1, V, E)
        
        X_out = self.lin_out(X_1)
        Y_out = self.lin_out(Y_1)

        return   X_out, self.X, E_com


In [ ]:
model, optimizer, G = initial(G, args, node_type, edge_type, mat_cu, mat_ua, mat_ca)

cuda:1
cuda:1


In [ ]:
#个体对比学习
def node_ssl_loss(args, node_embedding, edge_embedding):
    # pos and neg for the first graph
    user_node_embeddings = node_embedding[:args.user_number]
    com_node_embeddings = node_embedding[args.user_number:args.user_number+args.community_number]
    user_edge_embeddings = edge_embedding[:args.user_number]
    com_edge_embeddings = edge_embedding[args.user_number:args.user_number+args.community_number]
    user_node_embeddings_all = node_embedding[:args.user_number]
    com_node_embeddings_all = node_embedding[args.user_number:args.user_number+args.community_number]
    
    norm_user_emb1 = F.normalize(user_edge_embeddings)
    norm_user_emb2 = F.normalize(user_node_embeddings)
    norm_all_user_emb = F.normalize(user_node_embeddings_all)
    pos_score_user = torch.mul(norm_user_emb1, norm_user_emb2).sum(dim=1)
    ttl_score_user = torch.matmul(norm_user_emb1, norm_all_user_emb.transpose(0, 1))
    pos_score_user = torch.exp(pos_score_user / args.ssl_temp)
    ttl_score_user = torch.exp(ttl_score_user / args.ssl_temp).sum(axis=1)
    ssl_loss_user = -torch.log(pos_score_user / ttl_score_user).sum()

    norm_com_emb1 = F.normalize(com_edge_embeddings)
    norm_com_emb2 = F.normalize(com_node_embeddings)
    norm_all_com_emb = F.normalize(com_node_embeddings_all)
    pos_score_com = torch.mul(norm_com_emb1, norm_com_emb2).sum(dim=1)
    ttl_score_com = torch.matmul(norm_com_emb1, norm_all_com_emb.transpose(0, 1))
    pos_score_com = torch.exp(pos_score_com / args.ssl_temp)
    ttl_score_com = torch.exp(ttl_score_com/ args.ssl_temp).sum(axis=1)
    
    ssl_loss_com = -torch.log(pos_score_com / ttl_score_com).sum()
    ssl_loss = args.ssl_reg * (ssl_loss_user + ssl_loss_com)
    

    return ssl_loss

In [ ]:
def com_ssl_loss(args, com_embedding, hatt_embedding):

    com_embedding = com_embedding[args.user_number:args.user_number+args.community_number]
    norm_com_emb1 = F.normalize(hatt_embedding)
    norm_com_emb2 = F.normalize(com_embedding)
    norm_all_com_emb = F.normalize(hatt_embedding)
    pos_score_com = torch.mul(norm_com_emb1, norm_com_emb2).sum(dim=1)
    ttl_score_com = torch.matmul(norm_com_emb1, norm_all_com_emb.transpose(0, 1))
    pos_score_com = torch.exp(pos_score_com / args.ssl2_temp)
    ttl_score_com = torch.exp(ttl_score_com/ args.ssl2_temp).sum(axis=1)
    
    ssl_loss_com = -torch.log(pos_score_com / ttl_score_com).sum()
    ssl_loss = args.ssl2_reg*ssl_loss_com
    

    return ssl_loss

In [ ]:
def test(Z_1,test_label, user_com_edge_test):
    N_tr = args.train_user_number
    N_c = args.community_number
    N_te = args.test_user_number
    N_u = N_tr + N_te
    emb = Z_1[:N_u + N_c]
    test_predict_label=F.cosine_similarity(emb[user_com_edge_test[0]], emb[user_com_edge_test[1]])
    #test_predict_label = F.sigmoid(test_predict_label)
    test_predict_label=test_predict_label.detach().numpy()
    emb_norm = torch.norm(emb, dim=-1, keepdim=True)
    dot = torch.mm(emb, emb.t())
    dot_norm = torch.mm(emb_norm, emb_norm.t())
    sim = (dot / dot_norm)
    
    print('test_predict_label',test_predict_label)
    print('test_label',test_label)
    
            
    auc=sklearn.metrics.roc_auc_score(test_label,test_predict_label)
    ap = sklearn.metrics.average_precision_score(test_label, test_predict_label)
    print("ap:",ap)
    mea= np.mean(test_predict_label)
    acc=sklearn.metrics.accuracy_score(test_label,test_predict_label>mea)
    print('acc_test:',acc)
    return  auc, test_predict_label

In [ ]:
best_test_auc, test_auc = 0, 0
k = int(com_user_tensor_train.shape[1]/(1+args.negative_K))
for epoch in range(args.epochs):
    tic_epoch = time.time()
    model.train()
    optimizer.zero_grad()
    X, Y, Z= model()
    predict_label=F.cosine_similarity(X[com_user_tensor_train[0]], X[com_user_tensor_train[1]])
    #cos_loss = F.cosine_embedding_loss(com_user_tensor_train[0], com_user_tensor_train[1],)
    #predict_label = F.sigmoid(predict_label)
    #contrastive_loss = clloss(X_1, Y_1)
    if(epoch%10==0):
        print('EPOCH:',epoch,'####################################################')
        print('predict_label',predict_label)
        print('train_label',train_label)
        #auc=sklearn.metrics.roc_auc_score(pl, tl)
        #print("train auc", auc)
    cross_loss = F.binary_cross_entropy_with_logits(predict_label, train_label)
    ssl1_loss = node_ssl_loss(args, X, Y)
    ssl2_loss = com_ssl_loss(args, X, Z)
    loss= cross_loss*args.lam_1 + ssl1_loss*args.lam_2+ssl2_loss*args.lam_3
    print("Epoch:",epoch,"ssl2_loss",ssl2_loss)
    loss.backward()
    optimizer.step()

    if epoch>0:
        if epoch%2==0:
            auc, testl =test(X.cpu(), test_label, com_user_tensor_test)
            print("Epoch:",epoch, "AUC:",auc)
            if auc>best_test_auc:
                best_test_auc=auc
                print("best_test_auc:",best_test_auc)
            print("best_test_auc:",best_test_auc)